# Convert segmentation YOLO format to bbox YOLO format
This notebook is used to convert YOLO segmentation dataset into bounding box detection dataset

Functions

In [9]:
import os

def convert_seg_bb(directory): 
    # Loop through each .txt file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            
            # Read the content of the file
            with open(file_path, 'r') as file:
                contents = file.readlines()
            
            coords = []

            for content in contents:
                # Convert the segmentation labels to bounding box labels
                content = content.strip().split()
                class_label = content[0]
                x_values = []
                y_values = []
                
                for i in range(1, len(content), 2):
                    x_values.append(float(content[i]))
                    y_values.append(float(content[i + 1]))
                
                # Calculate the bounding box coordinates
                x1 = min(x_values)
                y1 = min(y_values)
                x2 = max(x_values)
                y2 = max(y_values)
                width = x2 - x1
                height = y2 - y1
                x = x1 + (width / 2)
                y = y1 + (height / 2)
                
                coords.append([class_label, x, y, width, height])

            with open(file_path, 'w') as file:
                for coord in coords:
                    file.write(f"{coord[0]} {coord[1]} {coord[2]} {coord[3]} {coord[4]}\n")

First, place a copy of the dataset into the `dataset` directory in the usual YOLO format:
- dataset/
    - train/
    - valid/

The cell below will **rewrite** the label files in the *train* directory
- NOTE: This rewrites the original segmentation files. Make backup of original dataset if needed.
- WARNING: Do not run this again after running it since the conversion has already occured. Doing so will result in unexpected output as it will try to convert the bbox

In [10]:
directory = r"C:\Users\Adrian\Desktop\projects\segment-anything-annotator\dataset\train\labels"
convert_seg_bb(directory)

The cell below will **rewrite** the label files in the *valid* directory
- NOTE: This rewrites the original segmentation files
- WARNING: Do not run this again after running it since the conversion has already occured. Doing so will result in unexpected output as it will try to convert the bbox

In [11]:
directory = r"C:\Users\Adrian\Desktop\projects\segment-anything-annotator\dataset\valid\labels"
convert_seg_bb(directory)

# Visualise the bounding box

Functions

In [12]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

def draw_bbox(image, bbox, color=(0,0,255), thickness=2):
    points = bbox

    # rescale points back to image dimenions  
    img_h = image.shape[0] 
    img_w = image.shape[1]
    points[0] = points[0] * img_w
    points[1] = points[1] * img_h
    points[2] = points[2] * img_w
    points[3] = points[3] * img_h


    x1, y1, w, h = points
    start_point = (int(x1 - w/2), int(y1 + h/2))
    end_point = (int(x1 + w/2), int(y1 - h/2))

    # draw bounding box on image
    cv2.rectangle(image, start_point, end_point, color, thickness)


# this function draws the existing bbox on the image
def draw_existing_bbox(image, path):
    with open(path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        parts = line.strip().split()
        bbox = [float(x) for x in parts[1:]]
        draw_bbox(image, bbox)

# this function draws the bbox on a set of images in a directory
def draw_bbox_on_images(image_dir, label_dir):
    for filename in os.listdir(image_dir):
        # get corresponding label file
        label_path = os.path.join(label_dir, filename.replace('.png', '.txt'))
        # read image
        image = cv2.imread(os.path.join(image_dir, filename))
        # draw existing bounding boxes
        draw_existing_bbox(image, label_path)
        # show image
        plt.imshow(image)
        plt.show()

Check bounding boxes for training set

In [1]:
image_dir = r"C:\Users\Adrian\Desktop\projects\segment-anything-annotator\dataset\train\images"
label_dir = r"C:\Users\Adrian\Desktop\projects\segment-anything-annotator\dataset\train\labels"

draw_bbox_on_images(image_dir, label_dir)

NameError: name 'draw_bbox_on_images' is not defined

Check bounding boxes for valid set

In [2]:
image_dir = r"C:\Users\Adrian\Desktop\projects\segment-anything-annotator\dataset\valid\images"
label_dir = r"C:\Users\Adrian\Desktop\projects\segment-anything-annotator\dataset\valid\labels"

draw_bbox_on_images(image_dir, label_dir)f

NameError: name 'draw_bbox_on_images' is not defined